In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
import pandas as pd

In [3]:
device  = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_dir = "models/kobart/best"

# model = AutoModelForCausalLM.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [16]:
# Beam
decode_params = {
    "num_beams": 5,
    "do_sample": False,
    "max_length": 256,
}

# Top P
decode_params = {
    "top_p": 0.8,
    "temperature": 0.8,
    "do_sample": True,
    "max_length": 256,
}


In [17]:
# Generate
query = "첫 번째, 저는 다양한 아르바이트 경험을 통해서 사람들과 함께 일하며 소통하는 것을 배웠습니다."

In [18]:
encoded = tokenizer(query, return_tensors = "pt")

input_ids = encoded["input_ids"].to(device)
attention_mask = encoded["attention_mask"].to(device)

with torch.no_grad():
    outs = model.generate(input_ids = input_ids, attention_mask = attention_mask, \
                        num_return_sequences = 1,
                        pad_token_id = tokenizer.pad_token_id,
                        **decode_params)

In [19]:
decoded = tokenizer.decode(outs[0], skip_special_tokens = True)
print("문장:", query)
print("생성 질문:",decoded)

문장: 첫 번째, 저는 다양한 아르바이트 경험을 통해서 사람들과 함께 일하며 소통하는 것을 배웠습니다.
생성 질문: 아르바 아르바이트 경험을 통해 사람들과 소통할 때 가장 힘들었던 점과 그 이유를 말씀해 주세요.
